In [ ]:
import sympy as sp
from matplotlib import pyplot as plt
import numpy as np

from sympy import init_printing
init_printing()

# Classical mechanics with SymPy
SymPy is great for automating a log of the machinery of classical Lagrangian and Hamiltonian mechanics. (Great enough for someone to build an entire [celestial mechanics package](https://celmech.readthedocs.io/en/latest/) on top of SymPy's capabilities for automating mechanics...)

In this example, we'll use sympy to generate equations of motion for a simple pendulum, turn these equations into callable functions, then numerically integrate the system.

## Simple Pendulum

We'll start by constructing the Lagrangian for a pendulum with mass $m$, length $l$, subject to a gravitational accelration $-g$. We'll use the angular displacement of the pendulum, $\theta$ as our generalized coordinate. 

### Lagrangian formulation

First we'll create our variables:

In [ ]:
theta_dot = sp.symbols(r"\dot{\theta}",real= True)
p,theta,t = sp.symbols("p,theta,t",real=True)
l,g,m = sp.symbols("l,g,m",positive=True)

Now we'll create expressions for the kinetic energy, $T$, and potential energy, $V$, which we then combine to get the Lagrangian, $L(\theta,\dot{\theta}) = T(\dot{\theta})-V(\theta)$ 

In [ ]:
KE = m * l*l*theta_dot*theta_dot/sp.S(2)
PE = m * g * l * (1 - sp.cos(theta))
lagrangian = KE - PE

In [ ]:
lagrangian

### Hamiltonian formulation
Recall that the Hamiltonian formulation is obtained form the Lagrangian formulation via a so-called Legendre transform, where we 

- (i) define a new momentum variable, $p = \frac{\partial }{\partial \dot{\theta}} L(\theta,\dot{\theta}) $
- (ii) Solve for $\dot\theta(p)$ by inverting the above equation 
- (iii)  Obtain the Hamiltonian as $H(\theta,p) = p\cdot\dot\theta(p) - L(\theta,\dot\theta(p))$

In [ ]:
to_solve = sp.Eq(p , sp.diff(lagrangian,theta_dot))
to_solve

In [ ]:
# get our solution
theta_dot_soln = sp.solve(to_solve,theta_dot)[0]
theta_dot_soln

Construct the Hamiltonian

In [ ]:
hamiltonian = (theta_dot * p - lagrangian).subs({theta_dot:theta_dot_soln})

In [ ]:
hamiltonian

### Hamiltonian equations of motion

Hamilton's equations are 
$$
\begin{eqnarray}
\frac{d}{dt}\theta &=& \partial_{p} H(\theta,p)
\\
\frac{d}{dt}p &=&  - \partial_{\theta} H(\theta,p)
\end{eqnarray}
$$

Get the right-hand side of these equations as a vector 

In [ ]:
flow = sp.Matrix([sp.diff(hamiltonian,p),-1* sp.diff(hamiltonian,theta)])

In [ ]:
flow

### Integration
We now have an ordinary differential equation for $(\theta,p)$ in symbolic form.

We could try to go about deducing an analytic solution, but who has time for learning elliptic integrals? Instead, we're going to turn our symbolic expressions into callable functions that we'll then feed to an ODE solver to generate a numerical solution.

We'll use scipy's `solve_ivp` function to integrate; let's see what we need to pass it:

In [ ]:
from scipy.integrate import solve_ivp

In [ ]:
help(solve_ivp)

In [ ]:
flow

Looking at the `solve_ivp` documentation, we need a function that takes the time and a state vector, $y$, possibly along with some other arguments, `args`, and returns $dy/dt$. 

SymPy's `lambidfy` can give us something like this:

In [ ]:
flow_func = sp.lambdify((t,theta,p,g,l,m),flow)

In [ ]:
Ng = 9.8 # numerical value for  grav. accelration
Nl = 1  # pendulum length, in meters
Nm = 0.5 # pendulum mass, in kg

theta0 = np.pi/2 # initial theta
p0 = 0 # initial p value

This has given us a function that we can call:

In [ ]:
flow_func(0,theta0,p0,Ng,Nl,Nm)

... that agree with substituting numerical values in our original expression:

In [ ]:
flow.subs({g:Ng,l:Nl,m:Nm,p:0,theta:np.pi/2})

Our `flow_func` isn't exactly in the form called for by `solve_ivp`, but we can easily make a wrapper that conforms to what `solve_ivp` asks for:

In [ ]:
fn = lambda t,y: flow_func(t,*y,Ng,Nl,Nm).reshape(-1)

In [ ]:
y0 = (theta0,p0)

fn(0,y0)

### Integrate

In [ ]:
Tfin = 10
Nout = 128
times = np.linspace(0,Tfin,Nout)

soln = solve_ivp(
    fn,
    (0,Tfin),
    y0,
    t_eval=times
)
soln

### Plot our results

In [ ]:
plt.plot(soln.t,soln.y[0])
plt.ylabel(r"$\theta$",fontsize=16)
plt.xlabel(r"Time",fontsize=16)

## Exercise 1

Lambidfy the Hamiltonian function, `hamiltonian`, defined about and use it to plot the pendulum's energy versus time.

In [ ]:
ham_func = sp.lambdify((theta,p,g,l,m),hamiltonian)

In [ ]:
E = ham_func(soln.y[0],soln.y[1],Ng,Nl,Nm)
dE = np.abs(E/np.mean(E)-1)
plt.plot(soln.t,dE)

## Getting the Jacobian for numerical integration

From the `solve_ivp` help documentation, we see that we can optionally pass the Jacobian of our flow as the `jac` parameter, which is used by some integration methods. In particular, implicit methods, are often improved by inclusion of Jacobian information.  

Furthermore, implicit methods usually have much better energy conservation properties for Hamiltonian systems. This is not so important for our simple pendulum example, but can often be important for more complicated systems you may encounter in astrodynamics.

In [ ]:
qp_vars = (theta,p)
flow_jacobian = sp.Matrix(2,2,lambda i,j: sp.diff(flow[i],qp_vars[j]))
flow_jacobian

Let's lambidfy again:

In [ ]:
jacobian_func  = sp.lambdify((t,theta,p,g,l,m),flow_jacobian)

In [ ]:
fn = lambda t,y: flow_func(t,*y,Ng,Nl,Nm).reshape(-1)
Dfn = lambda t,y: jacobian_func(t,*y,Ng,Nl,Nm)

In [ ]:
Dfn(0,y0)

In [ ]:
Tfin = 10
Nout = 128
times = np.linspace(0,Tfin,Nout)

soln = solve_ivp(
    fn,
    (0,Tfin),
    y0,
    t_eval=times,
    jac = Dfn,
    method='Radau'
)
soln

## Exercise 1.5
Using your lambdified Hamiltonian, confirm that the implicit Radau method that we just used has better energy conservation than our original solution


# The small angle approximation with `sympy.series`

When the maximum value of $\theta$ is small, the pendulum can be approximated as a harmonic oscillator. We'll use `sympy` to derive this harmonic oscillator solution.

First, we use `sympy.series` to get an approximate Hamiltonian:

In [ ]:
hamiltonian_small_theta = sp.series(hamiltonian,theta,x0=0,n=3)
hamiltonian_small_theta

We can get rid of the `bigO` term with the `removeO` method:

In [ ]:
hamiltonian_small_theta=hamiltonian_small_theta.removeO()
hamiltonian_small_theta

`sympy`'s `dsolve` method can solve (some) ODEs.
In order to use it, we need to represent our dynamical variables, $\theta$ and $p$, as `sympy.Functions` rather than simple `Symbols`.

In [ ]:
theta_fn, p_fn = sp.symbols("theta,p",cls=sp.Function)
theta_fn,p_fn

In [ ]:
type(theta),type(theta_fn)

After getting our equtaions of motion using Hamilton's equations, $\dot{\theta} = \partial_{p}H$ and $\dot{ p} = -\partial_{\theta}H$, we need to replace the occurences of our variables `theta` and `p` with their function versions, `theta_fn` and `p_fn` in order to have differntial equations in the form required by `dsolve`.

In [ ]:
subsrule = {p:p_fn(t),theta:theta_fn(t)}
subsrule

We apply Hamilton's equations and make the appropriate substitutions:

In [ ]:
eq1 =  sp.Eq(
    sp.Derivative(theta_fn(t),t),
    sp.diff(hamiltonian_small_theta,p).subs(subsrule)
)

eq2 = sp.Eq(
    sp.Derivative(p_fn(t),t),
    -sp.diff(hamiltonian_small_theta,theta).subs(subsrule)
)
eqs = (eq1,eq2)
eqs 

Set our initital conditions:

In [ ]:
ics = {theta_fn(0):sp.pi/2,p_fn(0):0}
ics

... and derive our soulution:

In [ ]:
small_angle_soln = sp.dsolve(eqs,[theta_fn(t),p_fn(t)],ics=ics)
small_angle_soln

### Exercise 1.75
Can you lambdify the small angle solution to compare it to our numerically integrated solution?

In [ ]:
small_angle_soln[0].rhs

## Exercise 2

Derive equations of motion directly from the Lagrangian, (i.e., the [Euler-Lagrange equations](https://en.wikipedia.org/wiki/Euler–Lagrange_equation)), and integrate them as we did with Hamilton's equations. 

## Exercise 3

- Can you derive and integrate Hamilton's equations of motion for the [double pendulum](https://en.wikipedia.org/wiki/Double_pendulum)? In case you need inspirartion for initial conditions, see [here](https://twitter.com/pendulum_bot?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor).

- Can you get a small-angle solution for this problem?